<a href="https://colab.research.google.com/github/VaishnaviNehare13/AIML/blob/main/AIML_Activity_Vaishnavi_Nehare_202301040037.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem statement : Discuss and Implement various strategies of constraint satisfaction problem for N

In [ ]:
# Problem statement : Discuss and Implement various strategies of constraint satisfaction problem for N

import time, random

# --- N-Queens CSP Class ---
class NQueensCSP:
    def __init__(self, N):
        self.N = N
        self.vars = list(range(N))  # columns (each queen placed in a column)
        self.domains = {v: list(range(N)) for v in self.vars}  # rows available for each column

    # Check if two queens conflict
    def conflict(self, var1, val1, var2, val2):
        if val1 == val2: return True  # same row
        if abs(val1 - val2) == abs(var1 - var2): return True  # same diagonal
        return False

# --- Consistency Check ---
def is_consistent(assignment, var, value, csp):
    # Check if placing queen at (var, value) is safe
    for v, val in assignment.items():
        if csp.conflict(v, val, var, value):
            return False
    return True

# --- Forward Checking ---
def forward_check(csp, var, value, domains, assignment):
    # Remove invalid values from future domains
    pruned = []
    for v in csp.vars:
        if v not in assignment and v != var:
            to_remove = [val for val in domains[v] if csp.conflict(var, value, v, val)]
            if to_remove:
                for r in to_remove: domains[v].remove(r)  # prune values
                pruned.append((v, to_remove))
            if not domains[v]:  # domain empty -> failure
                return False, pruned
    return True, pruned

# --- Backtracking Search ---
def backtrack(assignment, csp, domains, use_fc=False):
    # If all queens placed -> solution found
    if len(assignment) == len(csp.vars):
        return assignment

    # Select first unassigned column
    var = [v for v in csp.vars if v not in assignment][0]

    # Try all possible rows for this column
    for value in domains[var]:
        if is_consistent(assignment, var, value, csp):
            assignment[var] = value
            pruned = []

            # Apply forward checking if enabled
            if use_fc:
                ok, pruned = forward_check(csp, var, value, domains, assignment)
                if not ok:
                    # Undo pruning and try next value
                    for v, vals in pruned: domains[v].extend(vals)
                    del assignment[var]
                    continue

            # Recurse for next queen
            result = backtrack(assignment, csp, domains, use_fc)
            if result: return result

            # Undo changes if failed
            for v, vals in pruned: domains[v].extend(vals)
            del assignment[var]
    return None

# Solve using Backtracking (with/without Forward Checking)
def solve_backtracking(N, use_fc=False):
    csp = NQueensCSP(N)
    domains = {v: list(csp.domains[v]) for v in csp.vars}
    start = time.time()
    sol = backtrack({}, csp, domains, use_fc)
    return sol, time.time() - start

# --- Min-Conflicts Local Search ---
def min_conflicts(N, max_steps=50000):
    # Start with random queen positions
    cols = list(range(N))
    rows = [random.randrange(N) for _ in cols]

    # Count conflicts for a queen
    def conflicts(col, row):
        return sum(r == row or abs(r - row) == abs(c - col)
                   for c, r in enumerate(rows) if c != col)

    # Try to reduce conflicts iteratively
    for step in range(max_steps):
        conflicted = [c for c in cols if conflicts(c, rows[c]) > 0]
        if not conflicted: return rows, step  # solution found
        col = random.choice(conflicted)
        min_conf = min(conflicts(col, r) for r in range(N))
        candidates = [r for r in range(N) if conflicts(col, r) == min_conf]
        rows[col] = random.choice(candidates)
    return None, max_steps

# --- Demo Run ---
def run_demo(N=8):
    print(f"=== N-Queens  (N={N}) ===")

    # 1) Simple Backtracking
    sol, t = solve_backtracking(N)
    print(f"\n1) Backtracking: {sol}, Time={t:.4f}s")

    # 2) Backtracking with Forward Checking
    sol, t = solve_backtracking(N, use_fc=True)
    print(f"\n2) Backtracking + Forward Checking: {sol}, Time={t:.4f}s")

    # 3) Min-Conflicts Local Search
    sol, steps = min_conflicts(N)
    print(f"\n3) Min-Conflicts (Local Search): {sol}, Steps={steps}")

# Run demo
run_demo(8)


The N-Queens problem is a well-known Constraint Satisfaction Problem (CSP) in Artificial Intelligence, where the task is to place
𝑁
N queens on an
𝑁
×
𝑁
N×N chessboard such that no two queens attack each other. This means no two queens can share the same row, column, or diagonal. The challenge is to efficiently explore the large search space of possible arrangements and find one or more valid solutions.

In the given code, three different CSP strategies are implemented to solve the N-Queens problem. The first approach uses backtracking, where queens are placed column by column, and the algorithm backtracks whenever it encounters a conflict. The second approach enhances this by applying forward checking, which prunes invalid choices from future domains immediately after placing a queen, thereby avoiding dead ends early. The third approach applies a Min-Conflicts heuristic local search, which starts with a random configuration of queens and iteratively moves conflicted queens to positions with fewer conflicts until a solution is reached or the step limit is exceeded.

Together, these methods demonstrate different problem-solving strategies in CSP: backtracking provides completeness but can be slow, forward checking improves efficiency by reducing unnecessary exploration, and min-conflicts offers a fast heuristic-based solution well-suited for larger boards. This comparative implementation highlights the trade-offs between systematic search and heuristic local search in solving constraint satisfaction problems.